# Kuroshuto

- Blacken some cells of the grid.
- Cells with numbers must not be blackened. Black cells must not be orthogonally adjacent.
- Starting at a cell with the number N exactly one of the cells with distance N must be blackened.
- All white cells must form an orthogonally contiguous area.

----------

- 将网格的某些单元格涂黑;
- 带有数字的单元格不得变黑。黑色单元格不得正交相邻。
- 所有白细胞必须形成一个正交连续的区域。
- 从数字为 N 的单元格开始，必须对距离为 N 的单元格中的一个进行涂黑。（比如如果格子数为4，那么距离该格4格子的其他格子，有且仅有一个被涂黑）

![](https://www.janko.at/Raetsel/Kuroshuto/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Kuroshuto/Regeln-02.gif)

> Rules: https://www.janko.at/Raetsel/Kuroshuto/index.htm

TODO: Add SCIP solver!

In [7]:
from collections import deque
from itertools import chain
import gurobipy as gp
# ===== vsualzie ===== 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib.pyplot import MultipleLocator

In [3]:
def readGrid(path):
    with open(f"../assets/data/Kuroshuto/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("13_12x12")
    
    for g in grid:
        print(g)

['-', '-', '-', '-', '1', '-', '5', '1', '-', '-', '-', '-']
['4', '-', '-', '-', '5', '2', '-', '-', '-', '-', '-', '-']
['-', '-', '1', '-', '-', '-', '-', '2', '-', '-', '-', '-']
['-', '2', '-', '5', '-', '5', '-', '4', '4', '-', '-', '-']
['4', '-', '-', '5', '-', '-', '-', '-', '-', '-', '1', '-']
['-', '3', '-', '5', '2', '-', '2', '2', '-', '-', '-', '4']
['-', '-', '-', '-', '2', '4', '-', '-', '-', '-', '-', '8']
['-', '1', '-', '-', '4', '5', '-', '2', '1', '-', '2', '-']
['5', '-', '4', '-', '-', '-', '1', '2', '6', '5', '-', '3']
['-', '-', '-', '-', '-', '5', '-', '7', '4', '3', '-', '-']
['-', '-', '-', '1', '2', '-', '-', '-', '-', '-', '-', '-']
['1', '-', '-', '-', '-', '-', '-', '2', '2', '-', '3', '-']


In [20]:

def kuroshuto_solver(m, n, grid):
    kuroshuto = gp.Model("Kuroshuto")
    kuroshuto.Params.lazyConstraints = 1
    kuroshuto.update()
    x = {}
    for i in range(m):
        for j in range(n):
            x[i, j] = kuroshuto.addVar(
                    vtype = gp.GRB.BINARY,
                    obj = 1,
                    name = f"x[{i},{j}]"
                )
            if not grid[i][j] in ["-"]:
                kuroshuto.addConstr(x[i, j] == 0, name = f"fixed[{i},{j}")

    for i in range(m):
        for j in range(n):
            cand_pos = [] 
            if grid[i][j] in "0123456789":
                dist = int(grid[i][j])
                if i - dist >= 0 and grid[i - dist][j] not in "0123456789":
                    cand_pos.append((i - dist, j))
                if j - dist >= 0 and grid[i][j - dist] not in "0123456789":
                    cand_pos.append((i, j - dist))
                if i + dist < m and grid[i + dist][j] not in "0123456789":
                    cand_pos.append((i + dist, j))
                if j + dist < n and grid[i][j + dist] not in "0123456789":
                    cand_pos.append((i, j + dist))
                kuroshuto.addConstr(gp.quicksum([x[p] for p in cand_pos]) == 1, name = f"number_constr[{i},{j}]")
    
    for i in range(m):
        for j in range(n):
            if i + 1 < m:
                kuroshuto.addConstr(x[i, j] + x[i + 1, j] <= 1, name = f"adj_constr_1[{i},{j}]")
            if j + 1 < n:
                kuroshuto.addConstr(x[i, j] + x[i, j + 1] <= 1, name = f"adj_constr_2[{i},{j}]")
    
    
    
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for i in range(m)]
            for i in range(m):
                for j in range(n):
                    if int(x_sol[i, j]) == 0:
                        curr_grid[i][j] = 1
                
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white = sum(chain.from_iterable(grid))
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 1 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = [] # record cells that formulate the border: list(tuple)
                cells_inside_borders = [] # record all cells inside the border: list(tuple)
                cnt_white = 0
                while len(visited) < total_white:
                    start = find_unvisited_white()
                    if not start:
                        break
                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    current_inside_cells = [] # record cells in current border
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        current_inside_cells.append((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 0:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))
                    cells_inside_borders.append(current_inside_cells)
                    
                assert len(borders_cut) == len(cells_inside_borders)

                if cnt_white == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    return [], []
                # borders_cut.sort(key = lambda x: len(x), reverse = True)
                return borders_cut, cells_inside_borders

            borders_cut, cells_inside_borders = find_surrounding_black_cells(curr_grid)

            for borders, cells_inside in zip(borders_cut, cells_inside_borders):
                if len(cells_inside) < (m * n) // 2:
                    for (cell_x, cell_y) in cells_inside:
                        model.cbLazy(gp.quicksum(model._x[subx, suby] for (subx, suby) in borders ) <= len(borders) - 1 + model._x[cell_x, cell_y])

    
    kuroshuto._x = x
    kuroshuto.modelSense = gp.GRB.MAXIMIZE
    kuroshuto.optimize(border_elim)
    
    if (kuroshuto.status == gp.GRB.status.OPTIMAL):
        for i in range(m):
            for j in range(n):
                if grid[i][j] not in "0123456789":
                    if x[i, j].x > 1e-1:
                        print("X", end=" ")
                    else:
                        print("O", end=" ")
                else:
                    # print(grid[i][j], end=" ")
                    print("O", end=" ")
            print()
        print()

    else:
        print("Something Wrong!")
if __name__ == "__main__":
    m, n, grid = readGrid("1_8x8")
    kuroshuto_solver(m, n, grid)
    

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 170 rows, 64 columns and 308 nonzeros
Model fingerprint: 0xa33b5b54
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 170 rows and 64 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 11 available processors)

Solution count 1: 16 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.600000000000e+01, best bound 1.600000000000e+01, gap 0.0000%

User-callback calls 123, time in user-callback 0.